In [3]:
import pandas as pd
import pandas as pd
import re

In [1]:
paciente_sc = "PT_28"
ruta_single_cell = r'D:\Pitagoras\Spatia_seq\Dataset\PITAGORAS\Datos_Pedro\Single_cell\combined_df_sin_orphanVJ_leiden.xlsx'
ruta_spatial_TCRs = r"D:\Pitagoras\Spatia_seq\Dataset\PITAGORAS\Datos_Pedro\Datos_VDJ\GIN-71-3T-1.clonotypes.TRB.txt"
ruta_pos_counts = r"D:\Pitagoras\Spatia_seq\Dataset\PITAGORAS\Datos_Pedro\Clonotipos_SPATIAL\GIN_71_spatial_clonotype_counts"

ruta_pos_matrix_save = ruta_pos_counts + "_filtered.xlsx"
ruta_tabla = r"D:\Pitagoras\Spatia_seq\Dataset\PITAGORAS\Datos_Pedro\Clonotipos_SPATIAL"

In [4]:
sc_df = pd.read_excel(ruta_single_cell)

Quiero modificar el código para obtener un data frame de clonotipo con número de spots en el que aparece y los fenotipos del clonotipo del SingleCell

In [5]:
spatial_tcr = pd.read_table(ruta_spatial_TCRs)
pos_spatial_tcr = pd.read_table(ruta_pos_counts + '.csv', sep=';')
pos_spatial_tcr.set_index('Barcodes', inplace=True)
# sc_df = sc_df[sc_df['sample'] == paciente_sc]
# En ocsaoines tenemos clones con clone_id distinto que se repiten. 
# Tenemos que juntar los clones. 
numeric_cols = spatial_tcr.select_dtypes(include="number").columns
spatial_tcr['cloneId'] = [f'clone{ID}' for ID in spatial_tcr['cloneId'].values]
spatial_tcr_f = spatial_tcr.groupby(["nSeqCDR3", "aaSeqCDR3"])[numeric_cols].sum().reset_index()

spatial_tcr_f['cloneId'] = spatial_tcr_f['cloneId'].apply(lambda x: re.findall(r'clone\d+', x)[0])

# Data frame de los clones de spatial que se repiten en el single cell, con su anotación celular.
vdj_sc = sc_df[
              (sc_df['beta_1_junction'].isin(spatial_tcr_f['nSeqCDR3']) | (sc_df['beta_2_junction'].isin(spatial_tcr_f['nSeqCDR3'])))
              ][['beta_1_junction', 'beta_2_junction', 'Celltypist', 'leiden_names', 'ATLAS_TIL_KNN']]

# Como en GIN_63_spatial_clonotype_counts las columnas son cloneX y sólo queremos el número...
# pos_spatial_tcr.columns = resultado = ['Barcodes' if s == 'Barcodes' else int(s.replace('clone', '')) for s in pos_spatial_tcr]

# Creamos un diccionario para búsqueda rápida: {nSeqCDR3: cloneId}
cdr3_to_clone = dict(zip(spatial_tcr_f['nSeqCDR3'], spatial_tcr_f['cloneId']))

# Función que busca si beta_1 o beta_2 están en el diccionario y devuelve cloneId (o NaN)
def buscar_clone_id_beta1(row):
    beta = row['beta_1_junction']
    return cdr3_to_clone.get(beta)

def buscar_clone_id_beta2(row):
    beta = row['beta_2_junction']
    return cdr3_to_clone.get(beta)

# Aplicamos la función al DataFrame grande
vdj_sc['cloneId_beta1'] = vdj_sc.apply(buscar_clone_id_beta1, axis=1)
vdj_sc['cloneId_beta2'] = vdj_sc.apply(buscar_clone_id_beta2, axis=1)

# Ahora quiero mirar si en vdj hay CD4s y CD8s para poder pasarlo al dataframe pos_spatial_tcr 
# pos_spatial_tcr.set_index(keys='Barcodes', inplace=True) # Resustituyo el barcode como índice
# Como hay que unificar los clonotipos de TCR tanto del single como del Spatial
TCRs_single, TCRs_spatial = pd.concat([vdj_sc['cloneId_beta1'], vdj_sc['cloneId_beta2']]).dropna().unique(), pos_spatial_tcr.columns

cloneID = TCRs_single
clone_df = pd.DataFrame(0, index=cloneID, columns= vdj_sc['leiden_names'].unique())
for ID in cloneID:
    counts = vdj_sc.loc[(vdj_sc['cloneId_beta1'] == ID) | (vdj_sc['cloneId_beta2'] == ID)]['leiden_names'].value_counts()
    clone_df.loc[ID, counts.index] = counts.values

In [6]:
clone_df.to_excel(ruta_tabla + '/tabla_counts.xlsx')

Quiero hacer la relación de los clones que están en el spatial y en el single

In [7]:
spatial_tcr = pd.read_table(ruta_spatial_TCRs)
pos_spatial_tcr = pd.read_table(ruta_pos_counts + '.csv', sep=';').set_index('Barcodes')

numeric_cols = spatial_tcr.select_dtypes(include="number").columns
spatial_tcr['cloneId'] = [f'clone{ID}' for ID in spatial_tcr['cloneId'].values]
spatial_tcr_f = spatial_tcr.groupby(["nSeqCDR3", "aaSeqCDR3"])[numeric_cols].sum().reset_index()

full_mapping = dict(zip(spatial_tcr['cloneId'], spatial_tcr['nSeqCDR3']))

# Filtrar el diccionario para que solo tenga claves que están como columnas en pos_spatial_tcr
valid_mapping = {k: v for k, v in full_mapping.items() if k in pos_spatial_tcr.columns}

# Filtrar el DataFrame para mantener solo las columnas que están en el mapping
pos_spatial_tcr = pos_spatial_tcr[list(valid_mapping.keys())]

# Renombrar las columnas
pos_spatial_tcr = pos_spatial_tcr.rename(columns=valid_mapping)

# Agrupar columnas duplicadas (mismo nSeqCDR3) y sumarlas por fila
pos_spatial_tcr = pos_spatial_tcr.groupby(level=0, axis=1).sum()

# Crear el diccionario inverso {nSeqCDR3: primer cloneId que aparece}
reverse_mapping = (
    spatial_tcr.drop_duplicates(subset='nSeqCDR3', keep='first')
    .set_index('nSeqCDR3')['cloneId']
    .to_dict()
)

# Filtrar el mapping para las columnas actuales
reverse_mapping = {col: reverse_mapping[col] for col in pos_spatial_tcr.columns if col in reverse_mapping}

# Renombrar columnas
pos_spatial_tcr = pos_spatial_tcr.rename(columns=reverse_mapping)

C:\Users\pcastillor\AppData\Local\Temp\ipykernel_93300\2079515705.py:20: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  pos_spatial_tcr = pos_spatial_tcr.groupby(level=0, axis=1).sum()


In [8]:
pos_spatial_tcr.to_excel(ruta_pos_matrix_save)

Cambiamos la estrategia a la de arriba


In [ ]:
spatial_tcr = pd.read_table(ruta_spatial_TCRs)
pos_spatial_tcr = pd.read_table(ruta_pos_counts, index_col= False)

# sc_df = sc_df[sc_df['sample'] == paciente_sc]
# En ocsaoines tenemos clones con clone_id distinto que se repiten. 
# Tenemos que juntar los clones. 
numeric_cols = spatial_tcr.select_dtypes(include="number").columns
spatial_tcr['cloneId'] = [f'clone{ID}' for ID in spatial_tcr['cloneId'].values]
spatial_tcr_f = spatial_tcr.groupby(["nSeqCDR3", "aaSeqCDR3", "cloneId"])[numeric_cols].sum().reset_index()

# Data frame de los clones de spatial que se repiten en el single cell, con su anotación celular.
vdj_sc = sc_df[
              (sc_df['beta_1_junction'].isin(spatial_tcr_f['nSeqCDR3']) | (sc_df['beta_2_junction'].isin(spatial_tcr_f['nSeqCDR3'])))
              ][['beta_1_junction', 'beta_2_junction', 'Celltypist', 'leiden_names', 'ATLAS_TIL_KNN']]

# Como en GIN_63_spatial_clonotype_counts las columnas son cloneX y sólo queremos el número...
pos_spatial_tcr.columns = resultado = ['Barcodes' if s == 'Barcodes' else int(s.replace('clone', '')) for s in pos_spatial_tcr]

# Creamos un diccionario para búsqueda rápida: {nSeqCDR3: cloneId}
cdr3_to_clone = dict(zip(spatial_tcr_f['nSeqCDR3'], spatial_tcr_f['cloneId']))

# Función que busca si beta_1 o beta_2 están en el diccionario y devuelve cloneId (o NaN)
def buscar_clone_id_beta1(row):
    beta = row['beta_1_junction']
    return cdr3_to_clone.get(beta)

def buscar_clone_id_beta2(row):
    beta = row['beta_2_junction']
    return cdr3_to_clone.get(beta)

# Aplicamos la función al DataFrame grande
vdj_sc['cloneId_beta1'] = vdj_sc.apply(buscar_clone_id_beta1, axis=1)
vdj_sc['cloneId_beta2'] = vdj_sc.apply(buscar_clone_id_beta2, axis=1)

# Ahora quiero mirar si en vdj hay CD4s y CD8s para poder pasarlo al dataframe pos_spatial_tcr 
pos_spatial_tcr.set_index(keys='Barcodes', inplace=True) # Resustituyo el barcode como índice
# Como hay que unificar los clonotipos de TCR tanto del single como del Spatial
TCRs_single, TCRs_spatial = pd.concat([vdj_sc['cloneId_beta1'], vdj_sc['cloneId_beta2']]).dropna().unique(), pos_spatial_tcr.columns
TCRs_shared = TCRs_spatial.intersection(TCRs_single) # Estos son los TCRs compartidos

pos_spatial_tcr_f = pos_spatial_tcr[TCRs_shared] # Filtramos por los clones que aparecen en el archivo .txt

# Ahora cambiamos el nombre de la columna según su tipo celular CD4/CD8
cloneID = pos_spatial_tcr_f.columns
pos_spatial_ct = pos_spatial_tcr_f.copy()
for ID in cloneID:
    sts = vdj_sc.loc[(vdj_sc['cloneId_beta1'] == ID) | (vdj_sc['cloneId_beta2'] == ID)]['leiden_names'].describe().top
    try:
        cell_type = sts.split(sep='.')[0]
    except:
        cell_type = sts.split(sep='_')[0]
    
    pos_spatial_ct = pos_spatial_ct.rename(columns={ID:f'clone{ID}-'+cell_type})


In [ ]:
pos_spatial_ct.to_excel(ruta_pos_counts_save)